In [3]:
import pymysql
import csv
import datetime
import time
import configdbms

conn = pymysql.connect(
    host=configdbms.mysql["host"],
    port=configdbms.mysql["port"],
    user=configdbms.mysql["user"],
    passwd=configdbms.mysql["password"],
    db=configdbms.mysql["database"],
    autocommit=True
)

cursor = conn.cursor()

cursor.execute("DROP TABLE IF EXISTS ia626_article_keywords;")
cursor.execute("DROP TABLE IF EXISTS ia626_keywords;")
cursor.execute("DROP TABLE IF EXISTS ia626_articles;")

cursor.execute("""
CREATE TABLE ia626_articles (
    article_id INT AUTO_INCREMENT PRIMARY KEY,
    url TEXT,
    title VARCHAR(255),
    author VARCHAR(255),
    date DATETIME NULL,
    category VARCHAR(255),
    article_body LONGTEXT
);
""")

cursor.execute("""
CREATE TABLE ia626_keywords (
    keyword_id INT AUTO_INCREMENT PRIMARY KEY,
    keyword VARCHAR(255) UNIQUE
);
""")

cursor.execute("""
CREATE TABLE ia626_article_keywords (
    article_id INT,
    keyword_id INT,
    PRIMARY KEY (article_id, keyword_id),
    FOREIGN KEY (article_id) REFERENCES ia626_articles(article_id),
    FOREIGN KEY (keyword_id) REFERENCES ia626_keywords(keyword_id)
);
""")


date_formats = [
    "%Y-%m-%dT%H:%M:%S%z",
    "%Y-%m-%dT%H:%M:%S",
    "%a, %d %b %Y %H:%M:%S %z",
    "%Y-%m-%d %H:%M:%S",
    "%Y-%m-%d"
]

blocksize = 20
tokens = []
insert_time = 0
rows_processed = 0

with open("foxnews_articless.csv", "r", encoding="utf8") as f:
    reader = csv.reader(f)
    next(reader)

    for row in reader:
        url, title, author, date_raw, category, article_body = row

        dt_mysql = None
        if date_raw:
            for fmt in date_formats:
                try:
                    parsed = datetime.datetime.strptime(date_raw, fmt)
                    dt_mysql = parsed.strftime("%Y-%m-%d %H:%M:%S")
                    break
                except:
                    pass

        tokens.append((url, title, author, dt_mysql, category, article_body))

        if len(tokens) >= blocksize:
            start = time.time()
            cursor.executemany(
                """
                INSERT INTO ia626_articles (url, title, author, date, category, article_body)
                VALUES (%s, %s, %s, %s, %s, %s)
                """,
                tokens
            )
            insert_time += time.time() - start
            tokens = []

        rows_processed += 1
if tokens:
    start = time.time()
    cursor.executemany(
        """
        INSERT INTO ia626_articles (url, title, author, date, category, article_body)
        VALUES (%s, %s, %s, %s, %s, %s)
        """,
        tokens
    )
    insert_time += time.time() - start

print(f"Articles inserted! Time taken: {insert_time:.2f} seconds, Rows processed: {rows_processed}")
with open("semantic_tags.csv", "r", encoding="utf8") as f:
    reader = csv.reader(f)
    next(reader)

    for row in reader:
        article_id, tags_string = row

        tags = [t.strip() for t in tags_string.split(",") if t.strip()]

        for tag in tags:
            cursor.execute(
                "INSERT IGNORE INTO ia626_keywords (keyword) VALUES (%s)",
                (tag,)
            )

            cursor.execute(
                "SELECT keyword_id FROM ia626_keywords WHERE keyword = %s",
                (tag,)
            )
            keyword_id = cursor.fetchone()[0]

            cursor.execute(
                """
                INSERT IGNORE INTO ia626_article_keywords (article_id, keyword_id)
                VALUES (%s, %s)
                """,
                (article_id, keyword_id)
            )

cursor.close()
conn.close()

Articles inserted! Time taken: 4.63 seconds, Rows processed: 1000
